In [1]:
%matplotlib notebook
#from selenium import webdriver
import requests
import pandas as pd
from pandas import Timestamp
from pandas.tseries.offsets import DateOffset
import numpy as np
from StringIO import StringIO
import csv
import os
from dateutil import parser
import datetime as dt

FDIR=os.getenv('FDIR')
assert FDIR is not None

def downloadSymbols(symbols):

    fromday="01"
    frommonth="00"
    fromyear="2006"
    today="24"
    tomonth="03"
    toyear="2016"

    for symbol in symbols:
        r=requests.get("http://real-chart.finance.yahoo.com/table.csv?s={}&a={}&b={}&c={}&d={}&e={}&f={}&g=d&ignore=.csv".format(symbol, frommonth, fromday, fromyear, tomonth, today, toyear))
        if r.ok:
            f=open(FDIR+'/data/{}_{}_{}_{}_{}_{}_{}.csv'.format(symbol, frommonth, fromday, fromyear, tomonth, today, toyear),'w')
            f.write(r.content)
            f.close()
            
def loadSymbols(symbols):
    rootdir=FDIR+"/data"
    symbolFiles=os.listdir(rootdir)

    for symbolFile in symbolFiles:
        symbol=symbolFile.split('_')[0]
        item=symbols.get(symbol)
        if item is not None:
            item['file']=symbolFile
            
    print "Found {} files, loading symbols: {}".format(len(symbolFiles), ", ".join(symbols.keys()))

    print "Loading all symbols..."
    for symbol, pkg in symbols.items():
        fname=pkg['file']
        pkg['data']=pd.read_csv("/".join([rootdir,fname]), index_col=0, parse_dates=[0], infer_datetime_format=True)
        
def getPricePerShare(item, date):
    row=item['data'].loc[date]
    # Should use spread to choose random time during day for price
    #spread=row['high']-row['low']
    ## starting with worst case: buy highest price each day
    price=row['high']
    return price
        
def purchaseEquity(item, date, amount):
    price = getPricePerShare(item, date)
    sharesPurchased=np.floor(float(amount)/float(price))
    item['amountSpent'] += sharesPurchased*price
    item['sharesOwned'] += sharesPurchased
        
def buyIntoEquities(rs, date):
    for equity, item in rs.items():
        purchaseEquity(item, date, item['min'])
        
def getPurchaseAlloc(rs, date):
    purchaseAlloc={}
    if Timestamp(date).is_year_start:
        underAllocItems=[]
        total=0.0
        for equity, item in rs.items():
            price = getPricePerShare(item, date)
            total += item['sharesOwned']*price
        for equity, item in rs.items():
            price = getPricePerShare(item, date)
            alloc = item['sharesOwned']*price
            allocPercentage = alloc/total
            if allocPercentage > item['alloc']:
                purchaseAlloc[equity] = 0.0
                exceedAllocPercent += item['alloc']
            else:
                if allocPercentage < item['alloc']:
                    underAllocItems.append(equity)
                purchaseAlloc[equity] = item['alloc']
            
        distributeAlloc = exceedAllocPercent/len(underAllocItems)
        for equity in underAllocItems:
            purchaseAlloc[equity] += distributeAlloc

    else:
        for equity, item in rs.items():
            purchaseAlloc[equity] = item['alloc']
            
    print "Total purchase Alloc percent: {}".format(sum(purchaseAlloc.values()))
    print "Equities exceeding alloc: {}".format([a for a,i in purchaseAlloc.items() if i==0.0])
    print "Equities under alloc: {}".format(underAllocItems)


In [ ]:
rs={"FRESX":{'alloc':.2,'min':2500, 'sharesOwned':0, 'amountSpent':0, 'file':None}, 
    "FEMKX":{'alloc':.05,'min':2500, 'sharesOwned':0, 'amountSpent':0, 'file':None}, 
    "FBGRX":{'alloc':.3,'min':2500, 'sharesOwned':0, 'amountSpent':0, 'file':None}, 
    "FIGFX":{'alloc':.15,'min':2500, 'sharesOwned':0, 'amountSpent':0, 'file':None}}
startDate=dt.date(2006, 1, 3)
loadSymbols(rs)
#print rs['FRESX']['data'].loc().obj
months = pd.date_range(startDate, dt.date.today(), freq='BMS')
print months
for month in months:
    print month, Timestamp(month).is_year_start
purchasePerMonth=3
purchasDays=[7,15,22]
#purchases=purchasePerMonth*months
#skipDays=int(np.floor(float(len(rs['FRESX']['data']))/float(purchases)))
#print skipDays
#row = rs['FRESX']['data'][::skipDays]
#print row

In [3]:
rs=["FFFGX"]
rs={"FRESX":{'alloc':.2,'min':2500, 'sharesOwned':0, 'amountSpent':0}, 
    "FEMKX":{'alloc':.05,'min':2500, 'sharesOwned':0, 'amountSpent':0}, 
    "FBGRX":{'alloc':.3,'min':2500, 'sharesOwned':0, 'amountSpent':0}, 
    "FIGFX":{'alloc':.15,'min':2500, 'sharesOwned':0, 'amountSpent':0}}
startDate=dt.date(2006, 1, 3)
months=pd.bdate_range(startDate, dt.date.today(), freq='BMS')
loadSymbols(rs)
for symbol,pkg in rs.items():
    print "{}: {}".format(symbol, len(pkg['data']))
    
#TODO: need to assert we have data for each of these symbols or can run simulation

## Any two of the following three can be specified but not all three
## for now we will assume purchasePerMonth and monthsToBalance are the two specified
## purchasePerMonth is the number of purchases that can be made each month
## monthsToBalance is the number of months by which we want to have reached balanced allocation
purchasePerMonth=3; purchaseDays=[7,15,22]; maxPurchasePerMonth=None; monthsToBalance=3; 
baseInvestPerMonth=1000; balanceInvestPerMonth=0; purchasesTilBalance=0;
assert maxPurchasePerMonth is None and (purchasePerMonth is not None or purchaseDays is not None) and monthsToBalance is not None

## totalAtBalance is the smallest 100% total investment at which each asset class can meet its allocation %
#totalAtBalance=max([(1.0/v['alloc'])*v['min'] for s,v in rs.items()])

## remianingToBalance is the amount this asset class needs to still purchase in order to reach its allocation %
#for s,v in rs.items():
#    v['remianingToBalance']=totalAtBalance*(1.0-v['alloc'])

## amountAtEachPurchaseToBalance is the amount of this equity that needs to be purchased at each interval to reach
## the balanced allocation in the set amount of time
#for s,v in rs.items():
#    v['amountAtEachPurchaseToBalance']=totalAtBalance*(1.0-v['alloc'])

buyIntoEquities(rs, startDate)
purchaseAlloc={equity:item['alloc'] for equity, item in rs.items()}
for month in months:
    if Timestamp(month).is_year_start:
        purchaseAlloc=getPurchaseAlloc(rs, day)
    purchaseDates = [month + DateOffset(days=d) for d in purchaseDays]
    for day in purchaseDates:
        for equity, item in rs.items():
            amount=purchaseAlloc[equity]*float(baseInvestPerMonth)
            purchaseEquity(item, day, amount)


## Need to include bond payouts into assets
#downloadSymbols(rs.keys())



Found 4 files, loading symbols: FBGRX, FRESX, FEMKX, FIGFX
Loading all symbols...
FBGRX: 2594
FRESX: 2594
FEMKX: 2594
FIGFX: 2132


KeyError: 'high'